In [ ]:
from sqlalchemy.engine import URL
from sqlalchemy import Column, Integer, String, DateTime, Text, ForeignKey, Uuid, SmallInteger, DECIMAL
from sqlalchemy.orm import declarative_base
from datetime import datetime
from sqlalchemy.orm import sessionmaker
from home_security.data.postgresql.database import get_session, create_tables, seed_db
from home_security.data.postgresql.models import Household, Camera, ImageSet, IntrusionImage
from PIL import Image
from typing import List
from home_security.data.minio.minio import Minio
import uuid
from home_security.data.postgresql.database import Base, engine
Base.metadata.drop_all(engine) # this deletes all tables !!!
minio = Minio(endpoint="localhost:9000", access_key='homesecurity', secret_key='homesecuritybyjohn')


In [ ]:
session = get_session()
create_tables()
seed_db()

In [4]:
filename = "D01-1_20240605104950.jpg"
image = Image.open(f'../images/new_images/denning/{filename}')

In [6]:
seed_db()

In [ ]:
def add_image_set(images: List[Image.Image], camera_id, datetime_str):
    datetime_obj = datetime.strptime(datetime_str, "%Y%m%d%H%M%S")
    match filename[0]:
        case 'D':
            household_name='denning'
        case 'A':
            household_name = 'rectory'
        case _:
            raise Exception(f'Household code not found for recieved image {filename}')
        
    household_id = session.query(Household).filter(Household.name == household_name).all()[0].id
    if not session.get(Camera, camera_id):
        print(f'New camera feed detected at {household_name}, adding {camera_id} to DB!')
        new_camera = Camera(id=camera_id, household_id=household_id)
        session.add(new_camera)
        session.commit()
    else:
        print(f'Got new image for camera {camera_id} at household {household_name}')
    
    # Use this image set id to create an image set, add all the images as new Image and then upload the images to minio using the image ID as the key.
    imgset = ImageSet(received_date=datetime_obj, camera_id=camera_id, inferenced=False)
    session.add(imgset)
    session.commit()

    for image in images:
        image_uuid = uuid.uuid4()
        image_value = IntrusionImage(imageset_id=imgset.id, id=image_uuid)
        session.add(image_value)
        session.commit()
        # ADD IMAGE TO MINIO WITH THE IMAGE_UUID AS OBJECT NAME
        minio.add_image(image=image, image_uuid=str(image_uuid), bucket_name='intrusionimages')
        print(f'added image {image_value.id}, of set {image_value.imageset_id}')
        

In [12]:
add_image_set(images=[image], camera_id='D01', datetime_str='20250223144702')

Got new image for camera D01 at household denning
added image e3505bab-8b3a-4f29-bf11-c8f9a6ef32a2, of set 4
